# RAGAS - 基於知識圖譜的人工數據合成

## Single-hop 資料合成的概念說明

在 RAGAS 中，**Single-hop** 指的是：  
> 每一筆 Query–Answer Pair 僅依賴「單一知識節點（Node）」即可回答，  
> 不需要結合多個節點的資訊進行推理。

換句話說，Single-hop 並非完全沒有「關聯關係」，  
而是**生成 QA 時，只會選擇一個節點作為唯一事實來源**。

這類資料特別適合用來評估：
- 基礎檢索正確性（是否找得到對的段落）
- 回答是否忠實於單一來源（faithfulness）
- LLM 是否會憑空補完不存在的資訊

由於 Single-hop 的 Query–Answer Pair **必須完全來自同一個節點的內容**，  
因此節點本身需要包含「足夠完整的語意單位」。

若 chunk size 過小，容易導致：
- 問題問得出來，但答案資訊不足
- 或為了補齊答案而誘發 LLM hallucination

因此在 Single-hop 設計中，  
節點切分的原則通常是「一個節點 ≈ 一個可獨立回答的知識單位」。

In [ ]:
import os

os.chdir("../")

from src.ollama_connection import ragas_ollama

ragas_llm = ragas_ollama("gpt-oss:120b-cloud")

## 生成設定角色（Personas）

Persona 應該定義於測試集生成流程的最前段，
其功能不是影響資料選擇邏輯，
而是用來確保合成資料具備：

- 使用者場景的多樣性
- 問句語氣與表達方式的自然變化

Persona 影響的是「問題怎麼被問」，而不是「問題問什麼內容」。

例如：
- 「令狐沖學了什麼武功？」這個「內容」不會因 persona 改變
- 但新手可能會問：「令狐沖後來變強是因為學了什麼？」
- 資深讀者則可能問：「獨孤九劍的傳承是否代表劍宗思想的延續？」

這正是 persona 在合成資料中最重要的價值：  
**模擬不同使用者的提問視角，而非改變事實本身。**


若是不知道如何取得import路徑 (並不是所有的documentation都會寫得很詳細)，可以參照以下範例

```python
module = inspect.getmodule(SingleHopSpecificQuerySynthesizer.generate_query_reference_prompt.examples[0][1])
print(module.__name__) 
print(module.__file__)
```

我們創造五種類別的讀者:

In [ ]:
from textwrap import dedent

from ragas.testset.persona import Persona

persona_jinyong_new_reader = Persona(
    name="金庸新手讀者",
    role_description=(
        "剛開始接觸金庸小說，對人物、門派與故事背景不熟悉。"
        "常詢問基本角色介紹、小說之間的關係，以及簡單的情節概覽。"
        "需要清楚、直觀、不假設先備知識的說明。"
    ),
)

persona_jinyong_hardcore_fan = Persona(
    name="資深金庸迷",
    role_description=(
        "熟讀多部金庸小說，對角色關係、武功傳承與時間線非常熟悉。"
        "喜歡詢問細節設定、不同小說之間的連動、隱含線索與作者筆法。"
        "對模糊或不一致的回答容忍度低。"
    ),
)

persona_character_focused_reader = Persona(
    name="角色導向讀者",
    role_description=(
        "特別關注單一角色或少數角色的成長、性格與命運。"
        "常詢問角色的背景、武功來源、人際關係與結局。"
        "問題通常圍繞『這個人是誰、為什麼這樣做、最後怎麼樣』。"
    ),
)

persona_worldbuilding_researcher = Persona(
    name="武俠世界觀研究者",
    role_description=(
        "關心金庸小說中的門派體系、武功分類、江湖勢力與歷史脈絡。"
        "傾向提出比較型與結構型問題，例如不同門派的差異或武功傳承關係。"
        "希望回答具備系統性與一致性。"
    ),
)

persona_confused_casual_reader = Persona(
    name="記憶模糊的休閒讀者",
    role_description=(
        "以前讀過金庸小說，但對細節記憶模糊，容易混淆角色或情節。"
        "常提出『是不是…』『我記得好像…』這類確認型問題。"
        "需要糾錯與澄清，而不是假設提問者完全正確。"
    ),
)

personas_list = [
    persona_jinyong_new_reader,
    persona_jinyong_hardcore_fan,
    persona_character_focused_reader,
    persona_worldbuilding_researcher,
    persona_confused_casual_reader,
]

## Query-Answer Generation Prompt

這裡是RAGAS給予的範例

```python
persona = Persona(name="軟體工程師", role_description='專注於程式設計最佳實踐與系統設計。')
query_condition = QueryCondition(
                    persona=persona,
                    term='微服務',
                    query_style='正式',
                    query_length='中等',
                    context='微服務是一種架構風格，應用程式被設計為由多個鬆耦合的服務所組成。每個服務都是細粒度的，並專注於單一功能。',
                    llm_context=None
                  )
generative_query_answer = GeneratedQueryAnswer(
                    query='微服務在軟體架構中的目的為何？',
                    answer='微服務的設計目的是將應用程式結構化為由多個鬆耦合的服務所組成，每個服務專注於單一功能。'
                  )

examples = [(query_condition, generative_query_answer)]
```

在 `QueryCondition` 中，各欄位的角色如下：

- persona：決定提問者的語氣、關注點與提問方式
- term：強制要求問題中必須出現的關鍵詞
- query_style：控制語氣（正式、口語、分析型等）
- query_length：影響問題與答案的詳細程度
- context：**唯一可用的事實來源**
- llm_context：僅用來提示「問題型態」，不提供新事實


我們可以嘗試用ChatGPT/Gemini/Grok之類的服務來幫助我們仿照這個格式，產生新的範例。
譬如說，我們可以用剛剛產生的persona作為範例:

In [ ]:
from ragas.testset.synthesizers.single_hop.specific import SingleHopSpecificQuerySynthesizer
from ragas.testset.persona import Persona
from ragas.testset.synthesizers.single_hop.prompts import QueryAnswerGenerationPrompt, QueryCondition, GeneratedQueryAnswer

# QA Prompt
query_answer_generation_prompt = QueryAnswerGenerationPrompt(language='繁體中文/traditional Chinese')

**金庸新手讀者（入門、低先備）**

In [ ]:
query_condition_1 = QueryCondition(
    persona=persona_jinyong_new_reader,
    term="令狐沖",
    query_style="口語但清楚",
    query_length="中等",
    context=(
        "令狐沖是《笑傲江湖》的主要角色之一，"
        "原為華山派弟子，性格灑脫不拘禮法。"
    ),
    llm_context=None
)

generative_query_answer_1 = GeneratedQueryAnswer(
    query="令狐沖是誰？他在《笑傲江湖》裡扮演什麼角色？",
    answer=(
        "令狐沖是金庸小說《笑傲江湖》的主角，"
        "一開始是華山派的弟子，後來因為性格自由、不守門派成規，"
        "逐漸與師門產生衝突。他的經歷串起整部小說的主要情節。"
    )
)

**資深金庸迷（細節、設定一致性）**

In [ ]:
query_condition_2 = QueryCondition(
    persona=persona_jinyong_hardcore_fan,
    term="令狐沖",
    query_style="正式",
    query_length="偏長",
    context=(
        "令狐沖精通獨孤九劍，與風清揚的傳承關係，"
        "以及其與華山劍宗、氣宗之爭的關聯，是小說的重要設定。"
    ),
    llm_context=None
)

generative_query_answer_2 = GeneratedQueryAnswer(
    query="令狐沖學習獨孤九劍的過程，與華山劍宗傳承之間有何關聯？",
    answer=(
        "令狐沖的獨孤九劍來自風清揚，風清揚屬於華山劍宗一系。"
        "此一設定使令狐沖在無意間承接了劍宗精神，"
        "也加深了他與氣宗主流（岳不群一系）的思想與武學衝突，"
        "成為華山內部矛盾的具象化角色。"
    )
)

**角色導向讀者（人物動機與命運）**

In [ ]:
query_condition_3 = QueryCondition(
    persona=persona_character_focused_reader,
    term="令狐沖",
    query_style="敘事型",
    query_length="中等",
    context=(
        "令狐沖的性格灑脫、重情義，"
        "多次為朋友與愛人犧牲自身利益。"
    ),
    llm_context=None
)

generative_query_answer_3 = GeneratedQueryAnswer(
    query="為什麼令狐沖總是選擇違背門派規矩？這和他的性格有什麼關係？",
    answer=(
        "令狐沖重視情義勝過名分與規矩，"
        "他行事的出發點往往是『對人』而不是『對制度』。"
        "因此當門派規矩與他內心的道德判斷衝突時，"
        "他往往選擇後者，這也塑造了他坎坷卻自由的人生道路。"
    )
)

**武俠世界觀研究者（結構、比較）**

In [ ]:
query_condition_4 = QueryCondition(
    persona=persona_worldbuilding_researcher,
    term="獨孤九劍",
    query_style="分析型",
    query_length="偏長",
    context=(
        "獨孤九劍被視為一種重視破招而非內力的劍法，"
        "在金庸武學體系中具有高度特殊性。"
    ),
    llm_context=None
)

generative_query_answer_4 = GeneratedQueryAnswer(
    query="獨孤九劍在金庸武學體系中，與其他門派武功有何本質差異？",
    answer=(
        "獨孤九劍不強調內力積累或固定招式，"
        "而是以『破盡天下武學』為核心理念，"
        "屬於高度技巧導向的武學。"
        "相較少林、武當等重視內外兼修的體系，"
        "它代表了一種更自由、反體制的武學思路。"
    )
)


記憶模糊的休閒讀者（澄清、糾錯）

In [ ]:
query_condition_5 = QueryCondition(
    persona=persona_confused_casual_reader,
    term="令狐沖",
    query_style="確認型",
    query_length="短至中等",
    context=(
        "讀者容易將令狐沖與其他使用劍法的主角混淆，"
        "例如楊過或張無忌。"
    ),
    llm_context=None
)

generative_query_answer_5 = GeneratedQueryAnswer(
    query="我記得令狐沖是不是後來當了什麼門派掌門？還是我記錯了？",
    answer=(
        "你記得的方向有點接近，但不完全正確。"
        "令狐沖並沒有長期擔任主流門派的掌門，"
        "他的故事重點不在權位，而是在脫離門派束縛、"
        "追求個人自由與情感選擇。"
    )
)

因為語言模型的輸出品質高度依賴輸入品質，所以你有很好的文字功底的話，這會是你的優勢。

In [ ]:
query_answer_generation_prompt.instruction = dedent("""\
        根據指定條件（角色、術語、風格、長度）以及所提供的上下文，生成一個單跳（single-hop）的問題與答案。
        請確保答案完全忠實於上下文內容，僅使用直接來自所提供上下文的資訊。
        
        指示說明：
        
        1. 生成問題（Generate a Query）：
        根據上下文、角色（persona）、術語（term）、風格（style）與長度（length），建立一個符合該角色觀點並包含指定術語的問題。
        
        2. 生成答案（Generate an Answer）：
        僅使用所提供上下文中的內容來建構詳細答案。
        不得新增、推測或使用任何未包含於上下文中的資訊。
        
        3. 額外上下文（Additional Context）（若有提供）：
        若提供 llm_context，請將其作為問題類型與答案結構的指引
        （例如：比較型問題、操作說明型問題、應用型問題）。
        但答案內容仍必須完全來自提供的上下文。

        用繁體中文(traditional Chinese)進行回應。
""")


examples = [
    (query_condition_1, generative_query_answer_1),
    (query_condition_2, generative_query_answer_2),
    (query_condition_3, generative_query_answer_3),
    (query_condition_4, generative_query_answer_4),
    (query_condition_5, generative_query_answer_5),
]


query_answer_generation_prompt.examples = examples

## Theme Persona Matching Prompt

這裡是官方提供的範例:

```python
themes = ['包容性', '共情', '遠端工作']
personas = [Persona(name='HR Manager', role_description='專注於包容性與員工支持。'),
            Persona(name='Remote Team Lead', role_description='負責管理遠端團隊的溝通。')]
persona_themes_mapping = PersonaThemesMapping(mapping={"HR Manager": ['包容性', '共情'],
                                                       "Remote Team Lead": ['遠端工作', '共情']})
themes_personas_input = ThemesPersonasInput(themes=themes, personas=personas)

examples = [(persona_themes_mapping, themes_personas_input)]
```

我們可以基於已有的persona_list來產生examples,這樣會貼近我們的需求

In [ ]:
from ragas.testset.synthesizers.prompts import ThemesPersonasMatchingPrompt, ThemesPersonasInput, PersonaThemesMapping

theme_persona_matching_prompt = ThemesPersonasMatchingPrompt(language='繁體中文/traditional Chinese')
theme_persona_matching_prompt.instruction = dedent("""\
    給定一組主題（themes）以及包含其角色說明的角色（personas）清單，請根據每個角色的職責描述，將其與相關的主題進行關聯。
    用繁體中文(traditional Chinese)進行回應。
""")

themes = [
    "新手導覽",
    "角色解析",
    "情節概覽",
    "細節考據",
    "世界觀與門派體系",
    "跨書關聯",
    "澄清與糾錯"
]

persona_themes_mapping = PersonaThemesMapping(
    mapping={
        "金庸新手讀者": [
            "新手導覽",
            "角色解析",
            "情節概覽"
        ],
        "資深金庸迷": [
            "細節考據",
            "跨書關聯",
            "世界觀與門派體系"
        ],
        "角色導向讀者": [
            "角色解析",
            "情節概覽"
        ],
        "武俠世界觀研究者": [
            "世界觀與門派體系",
            "跨書關聯",
            "細節考據"
        ],
        "記憶模糊的休閒讀者": [
            "澄清與糾錯",
            "角色解析",
            "情節概覽"
        ],
    }
)

themes_personas_input = ThemesPersonasInput(
    themes=themes,
    personas=personas_list
)

# Examples 設定
examples = [
    (persona_themes_mapping, themes_personas_input)
]

theme_persona_matching_prompt.examples = examples

## 定義抽樣分布規則

根據某個節點上`property`，配合之前的theme_persona_matching_prompt 和 qa_answer_generation_prompt，決定數據合成分布。

In [ ]:
query_distibution = [
    (
        SingleHopSpecificQuerySynthesizer(llm=ragas_llm, 
                                          property_name="keyphrases", 
                                          name="keyphrases_synthesizer",
                                          theme_persona_matching_prompt=theme_persona_matching_prompt,
                                          generate_query_reference_prompt=query_answer_generation_prompt),
        1.0,
    ),
]

## 讀取知識圖

In [ ]:
from ragas.testset.graph import KnowledgeGraph, Node, NodeType
from langchain_community.document_loaders import DirectoryLoader

kg_single_hop = KnowledgeGraph.load("week_5/knowledge_graph_single_hop.json")

In [ ]:
# Jupyter 本身運行在一個事件迴圈中，有時與 RAGAS 內部的 asyncio 調度會產生衝突。

import nest_asyncio
nest_asyncio.apply()

In [ ]:
from ragas.testset import TestsetGenerator
from ragas.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings("BAAI/bge-m3")

generator = TestsetGenerator(
    llm=ragas_llm,
    embedding_model=embeddings,
    knowledge_graph=kg_single_hop,
    persona_list=personas_list,
)

single_hop_testset = generator.generate(testset_size=10, query_distribution=query_distibution)

In [ ]:
import hashlib

df = single_hop_testset.to_pandas()

def make_id(row):
    raw = f"{row['user_input']}|{row.get('reference', '')}"
    return hashlib.sha256(raw.encode("utf-8")).hexdigest()[:16]

df["testcase_id"] = df.apply(make_id, axis=1)

In [ ]:
df

**QueryLength**
Available options are: 
- LONG
- MEDIUM
- SHORT

**QueryStyle**
Available options are:
- MISSPELLED
- PERFECT_GRAMMAR
- POOR_GRAMMAR
- WEB_SEARCH_LIKE

> ⚠️ 注意：這裡的 QueryStyle enum  
> 與前面範例中使用的自然語言描述（如「正式」、「敘事型」）  
> **並非同一層級的設定**。

前者是 RAGAS 內部用於控制語言噪音的選項，  
後者則是 prompt engineering 層級的語意指引。


# 範例: 合成數據進行評估

In [ ]:
from ragas import Dataset

dataset = Dataset(name="test_dataset_synthesize", backend="local/csv", root_dir="week_5")

test_data = df.to_dict(orient='records')

for sample in test_data:
    dataset.append(sample)
dataset.save()

## 使用 LlamaIndex 完成 Response

**這是一個範例**

檢索策略: 使用第二周的 SentenceWindowNodeParser

In [ ]:
from glob import glob

from llama_index.core import SimpleDirectoryReader

reader = SimpleDirectoryReader(
    input_files=glob("week_5/data/*")
)

llama_index_docs = reader.load_data()

print(f"sizze of documents: {len(llama_index_docs)}")

建立VectorStoreIndex

In [ ]:
import faiss
from llama_index.core import VectorStoreIndex, StorageContext
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.vector_stores.faiss import FaissVectorStore
from llama_index.core.node_parser import SentenceSplitter, SentenceWindowNodeParser, MarkdownNodeParser
from llama_index.core.callbacks import LlamaDebugHandler, CallbackManager
from llama_index.core.postprocessor import MetadataReplacementPostProcessor

d = 1024 # 必須與 embedding model 的輸出維度一致

faiss_index = faiss.IndexFlatL2(d)
vector_store = FaissVectorStore(faiss_index=faiss_index)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-m3")

callback_manager = CallbackManager([LlamaDebugHandler()])

sentence_splitter = SentenceSplitter.from_defaults(chunk_size=128, chunk_overlap=0)

sw_parser = SentenceWindowNodeParser.from_defaults(
    window_size=2,
    window_metadata_key="window",
    original_text_metadata_key="original_text",
    sentence_splitter=sentence_splitter.split_text
)

raw_nodes = sw_parser.get_nodes_from_documents(llama_index_docs)

sw_index = VectorStoreIndex(
    raw_nodes,
    storage_context=storage_context,
    callback_manager=callback_manager,
    embed_model=embed_model
)

In [ ]:
len(storage_context.docstore.docs)

使用GPT-oss:120b-cloud 作為生成模型

In [ ]:
from src.ollama_connection import llama_index_ollama

llama_index_llm = llama_index_ollama(model="gpt-oss:120b-cloud", temperature=0)

建立query engine

In [ ]:
sw_query_engine = sw_index.as_query_engine(
    similarity_top_k=3,
    node_postprocessors=[
        MetadataReplacementPostProcessor(target_metadata_key="window")
    ],
    llm=llama_index_llm
)

測試能不能正常跑

In [ ]:
output = await sw_query_engine.aquery(df.iloc[0]['user_input'])

In [ ]:
output.response

In [ ]:
raw_text = output.response
clean_text = raw_text.replace("\u202f", " ")
print(clean_text)

In [ ]:
response.source_nodes[0].metadata['window']

批次生成結果

In [ ]:
import asyncio
from datetime import datetime

from ragas import Dataset

async def process_row(row):
    output = await sw_query_engine.aquery(row.user_input)
    return {
        "response": output.response,
        "reference": row.reference,
        "user_input": row.user_input,
        "retrieved_contexts": [
            node.metadata["window"] for node in output.source_nodes
        ],
        "testcase_id": row.testcase_id
    }

dataset = Dataset(name="test_dataset_synthetic_benchmark", backend="local/csv", root_dir="week_5")

BATCH_SIZE = 2  # tune this based on rate limits / memory

rows = list(df.itertuples(index=False))

time_start = datetime.now()

for i in range(0, len(rows), BATCH_SIZE):
    batch = rows[i:i + BATCH_SIZE]
    results = await asyncio.gather(
        *(process_row(row) for row in batch)
    )
    for item in results:
        dataset.append(item)

time_end = datetime.now()

print(time_end-time_start)

dataset.save()

In [ ]:
len(dataset)

進行量測實驗

In [ ]:
from pydantic import BaseModel
from ragas import experiment
from ragas.metrics.collections import Faithfulness, AnswerRelevancy, ContextPrecision, NoiseSensitivity

# Define experiment result structure
class ExperimentResult(BaseModel):
    faithfulness: float
    answer_relevancy: float


# 控制發送去Ollama的requests的數量: Semaphore 決定一次能進去跑的只有幾個
sem = asyncio.Semaphore(2)

async def limited(coro):
    async with sem:
        return await coro


# RAGAS 實驗：利用異步處理 (asyncio) 提升評估效率
@experiment(ExperimentResult)
async def run_evaluation(row):
    faithfulness = Faithfulness(llm=ragas_llm)
    answer_relevancy = AnswerRelevancy(llm=ragas_llm, embeddings=embeddings)
    context_precision = ContextPrecision(llm=ragas_llm)
    noise_sensitivity = NoiseSensitivity(llm=ragas_llm)
    
    faith_task = faithfulness.ascore(
        response=row.get("response"),
        retrieved_contexts=row.get("retrieved_contexts"),
        user_input=row.get("user_input"),
    )

    relevancy_task = answer_relevancy.ascore(
        user_input=row.get("user_input"),
        response=row.get("response"),
    )

    context_precision_task = context_precision.ascore(
        user_input=row.get("user_input"),
        reference=row.get("reference"),
        retrieved_contexts=row.get("retrieved_contexts"),
    )

    noise_sensitivity_task = noise_sensitivity.ascore(
        user_input=row.get("user_input"),
        reference=row.get("reference"),
        retrieved_contexts=row.get("retrieved_contexts"),
        response=row.get("response"),
    )
    
    # asyncio.gather 讓大家一起排隊跑
    (
        faith_result,
        relevancy_result,
        context_precision_result,
        noise_sensitivity_result,
    ) = await asyncio.gather(
        limited(faith_task),
        limited(relevancy_task),
        limited(context_precision_task),
        limited(noise_sensitivity_task),
    )
    
    return ExperimentResult(
        testcase_id=row.get("testcase_id"),
        faithfulness=faith_result.value,
        answer_relevancy=relevancy_result.value,
        context_precision=context_precision_result.value,
        noise_sensitivity=noise_sensitivity_result.value
    )

In [ ]:
exp_results = await run_evaluation.arun(dataset, name="benchmark_evaluation")

In [ ]:
exp_results.to_pandas()

In [ ]:
exp_results.to_pandas().to_csv("week_6/test_dataset_synthesized_benchmark_evaluation.csv", index=False)

# Multi-hop 數據合成

## Multi-hop 資料合成的核心邏輯

Multi-hop 的目標是：  
> 生成一個**必須同時結合兩個以上節點資訊**才能回答的問題。

RAGAS 在預設實作中，  
是透過「節點之間是否存在可量化的關聯關係」來決定是否能形成 multi-hop 問題。


從原代碼了解邏輯:

1. 透過選擇relationship的type來決定如何連結節點
```python
name: str = "multi_hop_specific_query_synthesizer"
property_name: str = "entities"
relation_type: str = "entities_overlap"
relation_overlap_property: str = "overlapped_items"
theme_persona_matching_prompt: PydanticPrompt = ThemesPersonasMatchingPrompt()
generate_query_reference_prompt: PydanticPrompt = QueryAnswerGenerationPrompt()

def get_node_clusters(self, knowledge_graph: KnowledgeGraph) -> t.List[t.Tuple]:
    """Identify clusters of nodes based on the specified relationship condition."""
    node_clusters = knowledge_graph.find_two_nodes_single_rel(
        relationship_condition=lambda rel: rel.type == self.relation_type
    )
    logger.info("found %d clusters", len(node_clusters))
    return node_clusters

```

2. 需要有 類似於 "overlapped_items"的項，而且必須要是文字
```python

for triplet in triplets:
    if len(scenarios) < n:
        node_a, node_b = triplet[0], triplet[-1]
        overlapped_items = []
        overlapped_items = triplet[1].properties[self.relation_overlap_property]
```
在 RAGAS中，`OverlapScoreBuilder` 符合這項需求

控制參數:
    - threshold：「重疊程度要高到多少，才算有意義？」（最低接受門檻）
    - distance_threshold：「兩個 keyphrase 多像，才算同一個？」（相似度判定門檻）

需要注意的是，`OverlapScoreBuilder` 本質上仍是基於「文字層級的相似度計算」，譬如說 水果`西瓜`和藝人`胡瓜`，而非語意理解。

因此即使兩個詞在語意上毫無關聯，
只要在 embedding space 中距離接近，就可能被視為 overlap。

這也是為什麼在實務上，  
Multi-hop 合成後往往還需要**第二層的語意過濾機制**。

In [ ]:
from ragas.run_config import RunConfig
from ragas.testset.transforms import apply_transforms, OverlapScoreBuilder 

overlap_relationship_builder = OverlapScoreBuilder(property_name="keyphrases", new_property_name="overlap_score",
                                                   threshold=0.01, distance_threshold=0.5)

transforms = [
    overlap_relationship_builder
]

run_config = RunConfig(
    max_workers=3,   # 👈 控制同時 LLM 請求數
    timeout=120,
    max_retries=5
)

apply_transforms(kg_single_hop, transforms, run_config=run_config)

In [ ]:
results = kg_single_hop.find_two_nodes_single_rel(
            relationship_condition=lambda rel: (
                True if rel.type == "keyphrases_overlap" else False
            )
        )

len(results)

In [ ]:
query_answer_generation_prompt = QueryAnswerGenerationPrompt(language='繁體中文/traditional Chinese')

persona = Persona(name="歷史學家", role_description='專注於重大的科學里程碑及其對全球的影響。')
query_condition = QueryCondition(
                    persona=persona,
                    themes=['相對論', '實驗驗證'],
                    query_style='正式',
                    query_length='中等',
                    context=[
                              '<1-hop> 阿爾伯特・愛因斯坦提出了相對論，引入了時空的概念。',
                              '<2-hop> 1919 年的日全食期間，透過觀測證實了重力會使光線彎曲，從而支持了愛因斯坦的理論。'
                            ],
                    llm_context=None
                  )
generative_query_answer = GeneratedQueryAnswer(
                    query='在 1919 年的日全食期間，相對論的實驗驗證是如何完成的？',
                    answer='相對論的實驗驗證是在 1919 年的日全食期間，透過證實重力會使光線發生彎曲來完成的，這一觀測結果支持了愛因斯坦在相對論中提出的時空概念。'
                  )

examples = [(query_condition, generative_query_answer)]

query_answer_generation_prompt.instruction = dedent("""
根據指定的條件（角色設定、主題、風格、長度）以及所提供的背景內容，生成一個多跳（Multi-hop）問題與答案。主題代表一組從背景中擷取或生成的片語，用以突顯所選背景內容適合用於多跳問題的原因。請確保問題中明確納入這些主題。

### 說明：
1. 生成多跳問題：  
   使用提供的背景段落與主題來形成一個需要結合多個段落資訊的問題（例如 `<1-hop>` 與 `<2-hop>`）。請確保問題中明確納入一個或多個主題，並反映其與背景內容的相關性。

2. 生成答案：  
   僅使用提供的背景內容來產生一個詳細且忠實於原文的答案。避免加入任何未直接出現在背景中，或無法從背景合理推論出的資訊。

3. 多跳背景標籤：  
   - 每個背景段落皆標註為 `<1-hop>`、`<2-hop>` 等。  
   - 請確保問題至少使用來自兩個不同段落的資訊，並且在意義上將它們有機地連結起來。

4. 額外背景（如有提供）：  
   若提供 `llm_context`，請將其作為指引，用來決定要生成的問題類型（例如：比較型問題、因果關係問題、應用導向問題）以及答案的結構方式。但仍需確保所有內容僅來自提供的背景資料。
""")

query_answer_generation_prompt.examples = examples

In [ ]:
from ragas.testset.synthesizers.multi_hop.specific import MultiHopSpecificQuerySynthesizer

synthesizer = MultiHopSpecificQuerySynthesizer(generate_query_reference_prompt=query_answer_generation_prompt,
                                 llm=ragas_llm,
                                 property_name='keyphrases',
                                 relation_type='keyphrases_overlap',
                                 relation_overlap_property="overlapped_items"
                                 )

query_distibution = [
    (
        synthesizer, 1.0,
    ),
]

generator = TestsetGenerator(
    llm=ragas_llm,
    embedding_model=embeddings,
    knowledge_graph=kg_single_hop,
    persona_list=personas_list,
)

multi_hop_testset = generator.generate(testset_size=10, query_distribution=query_distibution)

In [ ]:
df = multi_hop_testset.to_pandas()

def make_id(row):
    raw = f"{row['user_input']}|{row.get('reference', '')}"
    return hashlib.sha256(raw.encode("utf-8")).hexdigest()[:16]

df["testcase_id"] = df.apply(make_id, axis=1)

In [ ]:
df

### 節點過濾

基礎MultiHopSpecificQuerySynthesizer是透過節點之間的重疊程度來決定是否關聯，但並不考慮node內的巨觀內容。

我們可以嘗試透過節點之間的cosine similarities來刪除低相似的節點。

In [ ]:
threshold = 0.7
filtered_relationships = []

import numpy as np

def calculate_cosine(vec_a, vec_b):
    """
    Calculates the cosine similarity between two vectors.
    Returns a float between -1.0 and 1.0.
    """
    # Convert to numpy arrays if they aren't already
    a = np.array(vec_a)
    b = np.array(vec_b)
    
    # Calculate the dot product
    dot_product = np.dot(a, b)
    
    # Calculate the norms (magnitudes)
    norm_a = np.linalg.norm(a)
    norm_b = np.linalg.norm(b)
    
    # Avoid division by zero if a vector is all zeros
    if norm_a == 0 or norm_b == 0:
        return 0.0
    
    return dot_product / (norm_a * norm_b)

for rel in kg_single_hop.relationships:
    # Extract embeddings
    vec_a = rel.source.properties['summary_embedding']
    vec_b = rel.target.properties['summary_embedding']
    
    # Calculate similarity
    sim = calculate_cosine(vec_a, vec_b) 
    
    # Only keep if it meets the criteria
    if sim >= threshold:
        rel.properties['cosine_sim'] = sim # Store it for later!
        filtered_relationships.append(rel)

In [ ]:
# Create a fresh KG instance
new_kg = KnowledgeGraph()

# Add the filtered relationships
# Note: Most frameworks automatically add the associated nodes when a relationship is added
new_kg.relationships = filtered_relationships

In [ ]:
new_kg.nodes = list(set([rel.source for rel in filtered_relationships] + 
                        [rel.target for rel in filtered_relationships]))

In [ ]:
results = new_kg.find_two_nodes_single_rel(
            relationship_condition=lambda rel: (
                True if rel.type == "keyphrases_overlap" else False
            )
        )

results[:5]

In [ ]:
len(results)

我們可以看到關聯的節點大幅度縮減: 從78701 -> 2946。

In [ ]:
generator = TestsetGenerator(
    llm=ragas_llm,
    embedding_model=embeddings,
    knowledge_graph=new_kg,
    persona_list=personas_list,
)

multi_hop_testset = generator.generate(testset_size=10, query_distribution=query_distibution)

In [ ]:
multi_hop_testset.to_pandas()